In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#path
path = r'C:\Users\JocGa\Instacart Basket Analysis'

In [3]:
#import dataframes
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_update2.pkl'))

In [4]:
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [5]:
#review df_customers
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
df_customers.shape

(206209, 10)

# Rename and Drop columns

In [7]:
#drop columns
df_customers = df_customers.drop(columns = ['First Name', 'Surnam'])

In [8]:
#renaming columns
df_customers.rename(columns = {'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'n_dependants' : 'number_dependants', 'fam_status' : 'family_status' }, inplace = True)

In [9]:
#review new dataframe
df_customers.head()

,user_id,gender,state,age,date_joined,number_dependants,family_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


# Data Quality and Consistency check

In [10]:
#Descriptive Statistics
df_customers.describe()

,user_id,age,number_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


#column user_id should be a string not an int64

In [11]:
#check for mixed-type columns
for col in df_customers.columns.tolist():
    weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_customers[weird]) > 0:
        print(col)

#shows no mixed-type in columns

In [12]:
#check dataset type
df_customers.dtypes

user_id               int64
gender               object
state                object
age                   int64
date_joined          object
number_dependants     int64
family_status        object
income                int64
dtype: object

#change user_id dtype to int64 to string (o)

In [13]:
#change variable data type
df_customers['user_id'] = df_customers['user_id'].astype('str')

In [14]:
#check dataset type for change
df_customers.dtypes

user_id              object
gender               object
state                object
age                   int64
date_joined          object
number_dependants     int64
family_status        object
income                int64
dtype: object

In [15]:
#Descriptive Statistics
df_customers.describe()

,age,number_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


#no outliers

In [16]:
#finding missing values
df_customers.isnull().sum()

user_id              0
gender               0
state                0
age                  0
date_joined          0
number_dependants    0
family_status        0
income               0
dtype: int64

#no missing values in columns

In [17]:
#finding duplicates
df_customers[df_customers.duplicated()]

,user_id,gender,state,age,date_joined,number_dependants,family_status,income


#no duplicates

# Combine Dataframe

In [18]:
#dataframe number of rows and columns
ords_prods_merge.shape

(32404859, 24)

In [19]:
#dataframe number of rows and columns
df_customers.shape

(206209, 8)

#not a full match, different amount of rows and columns

In [20]:
#overview of dataframe
ords_prods_merge.head(2)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_prior_orders,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,both,...,Mid-range product,Regularly busy,Regularly busys,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [21]:
#overview of dataframe
df_customers.head(2)

,user_id,gender,state,age,date_joined,number_dependants,family_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285


In [22]:
#change variable data type
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')

In [23]:
#checking change datatype
ords_prods_merge.dtypes

order_id                     int64
user_id                     object
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
_merge                    category
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
average_price              float64
spending_flag               object
median_prior_orders        float64
order_frequency_flag        object
dtype: object

In [24]:
#merging datasets
df_ords_prods_custm = pd.merge(ords_prods_merge, df_customers, on = 'user_id', how = 'outer')

#using the 'outer' will combine all the observations and show you the real merge rate

In [25]:
df_ords_prods_custm

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,spending_flag,median_prior_orders,order_frequency_flag,gender,state,age,date_joined,number_dependants,family_status,income
0,2539329,1,1,2,8,NaN,196,1,0,both,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,both,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,both,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,both,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,both,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,156685,106143,26,4,23,5.0,19675,1,1,both,...,High spender,7.0,Frequent customer,Male,Hawaii,25,5/26/2017,0,single,53755
32404855,484769,66343,1,6,11,NaN,47210,1,0,both,...,Low spender,30.0,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151
32404856,1561557,66343,2,1,11,30.0,47210,1,1,both,...,Low spender,30.0,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151
32404857,276317,66343,3,6,15,19.0,47210,1,1,both,...,Low spender,30.0,Non-frequent customer,Female,Tennessee,22,9/12/2017,3,married,46151


In [26]:
df_ords_prods_custm['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

#new dataframe has a full match

In [27]:
#dataframe #rows, and columns
df_ords_prods_custm.shape

(32404859, 31)

# Export data to pkl

In [28]:
#export dataframe
df_ords_prods_custm.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers.pkl'))